In [1]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.runnables import RunnablePassthrough,RunnableLambda, Runnable, RunnableParallel,RunnableConfig
from langchain_core.messages import AIMessage
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.tools.render import render_text_description
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate,SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from operator import itemgetter
from langchain.embeddings import SentenceTransformerEmbeddings
import json
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS,Chroma
from operator import itemgetter
import time
import grandalf

C:\Users\MSI 1\.conda\envs\oaivenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
load_dotenv(find_dotenv("D:\LLM Courses\Master Langchain Udemy\.env"))

True

In [4]:
llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash")
# llm=ChatOpenAI(model="gpt-3.5-turbo")

In [4]:
prompt=ChatPromptTemplate.from_template(
    template="Tell me a Joke about {topic}"
)

In [5]:
chain=prompt|llm

<h2> Input Schema </h2>

<h5>The Input Schema of the chain is the input schema of its first part, the prompt</h5>

In [8]:
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}},
 'required': ['topic']}

In [9]:
prompt.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}},
 'required': ['topic']}

In [11]:
# Not the same as the the chain and the prompt
llm.input_schema.schema()

{'title': 'ChatGoogleGenerativeAIInput',
 'anyOf': [{'type': 'string'},
  {'$ref': '#/definitions/StringPromptValue'},
  {'$ref': '#/definitions/ChatPromptValueConcrete'},
  {'type': 'array',
   'items': {'anyOf': [{'$ref': '#/definitions/AIMessage'},
     {'$ref': '#/definitions/HumanMessage'},
     {'$ref': '#/definitions/ChatMessage'},
     {'$ref': '#/definitions/SystemMessage'},
     {'$ref': '#/definitions/FunctionMessage'},
     {'$ref': '#/definitions/ToolMessage'}]}}],
 'definitions': {'StringPromptValue': {'title': 'StringPromptValue',
   'description': 'String prompt value.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'type': {'title': 'Type',
     'default': 'StringPromptValue',
     'enum': ['StringPromptValue'],
     'type': 'string'}},
   'required': ['text']},
  'ToolCall': {'title': 'ToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'object'},

<h3>Output Schema</h3>

<h5>The Output Schema of the chain is the output schema of its last part, in this case a ChatModel, which outputs a ChatMessage</h5>

In [12]:
chain.output_schema.schema()

{'title': 'ChatGoogleGenerativeAIOutput',
 'anyOf': [{'$ref': '#/definitions/AIMessage'},
  {'$ref': '#/definitions/HumanMessage'},
  {'$ref': '#/definitions/ChatMessage'},
  {'$ref': '#/definitions/SystemMessage'},
  {'$ref': '#/definitions/FunctionMessage'},
  {'$ref': '#/definitions/ToolMessage'}],
 'definitions': {'ToolCall': {'title': 'ToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'object'},
    'id': {'title': 'Id', 'type': 'string'},
    'type': {'title': 'Type', 'enum': ['tool_call'], 'type': 'string'}},
   'required': ['name', 'args', 'id']},
  'InvalidToolCall': {'title': 'InvalidToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'string'},
    'id': {'title': 'Id', 'type': 'string'},
    'error': {'title': 'Error', 'type': 'string'},
    'type': {'title': 'Type',
     'enum': ['invalid_tool_call'],
     'type

In [13]:
llm.output_schema.schema()

{'title': 'ChatGoogleGenerativeAIOutput',
 'anyOf': [{'$ref': '#/definitions/AIMessage'},
  {'$ref': '#/definitions/HumanMessage'},
  {'$ref': '#/definitions/ChatMessage'},
  {'$ref': '#/definitions/SystemMessage'},
  {'$ref': '#/definitions/FunctionMessage'},
  {'$ref': '#/definitions/ToolMessage'}],
 'definitions': {'ToolCall': {'title': 'ToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'object'},
    'id': {'title': 'Id', 'type': 'string'},
    'type': {'title': 'Type', 'enum': ['tool_call'], 'type': 'string'}},
   'required': ['name', 'args', 'id']},
  'InvalidToolCall': {'title': 'InvalidToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'string'},
    'id': {'title': 'Id', 'type': 'string'},
    'error': {'title': 'Error', 'type': 'string'},
    'type': {'title': 'Type',
     'enum': ['invalid_tool_call'],
     'type

<h3>Stream</h3>

In [20]:
for s in chain.stream(input={'topic':'bear'}):
    print(s.content,end="",flush=True)

Why don't they play poker in the forest? 

Because there's too many cheetahs! 


<h3>Invoke</h3>

In [22]:
chain.invoke(input={"topic":"bear"}).content

'Why did the bear say no to dessert? \n\nBecause he was stuffed! 🐻 \n'

<h3>Batch</h3>

In [24]:
chain.batch(inputs=[
    {"topic":"bear"},
    {"topic":"Virat Kohli"}
])

[AIMessage(content='Why did the bear say no to dessert?\n\nBecause he was already stuffed! 🐻 \n', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-b4925534-6e32-4c8d-ae19-e47eab51ac82-0'),
 AIMessage(content="Why did Virat Kohli get a job at the bank? \n\nBecause he's a master at chasing runs! 🏏💰 \n", response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_

In [25]:
# Setting Concurrency

chain.batch(
    inputs=[
    {"topic":"bear"},
    {"topic":"Virat Kohli"},
    ],
    config={
        "max_concurrency":5
    }
)

[AIMessage(content="Why don't they play poker in the forest? \n\nBecause there's too many cheetahs! \n", response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-0fdad0bc-57d8-4435-b8d0-4be640bcd1bc-0'),
 AIMessage(content="Why did Virat Kohli get a job at the library?\n\nBecause he's a real bookworm, and he's always chasing runs! 🏏📚 \n", response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked

<h3>Async Stream</h3>

In [31]:
async for s in chain.astream(input={'topic':'bear'}):
    print(s.content,end="",flush=True)

Why don't they play poker in the forest?

Because there's too many cheetahs! 


<h3> Async Invoke </h3>

In [42]:
await chain.ainvoke(input={"topic":"Rohit Sharma"})

AIMessage(content="Why did Rohit Sharma get a job at a bank?\n\nBecause he's a master at handling pressure and knows how to hit sixes! 🏏💰 \n", response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'LOW', 'blocked': False}]}, id='run-204ca80e-f48d-4006-8bd4-0fbded573d82-0')

<h3>Async Batch </h3>

In [43]:
await chain.abatch(inputs=[{"topic":"Sachin Tendulkar"},{"topic":"Shane Warne"}])

[AIMessage(content="Why did Sachin Tendulkar always carry a spare bat to the cricket ground?\n\nBecause he knew he'd be hitting a century! 🏏😄 \n", response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-204719cd-0947-459b-b34e-226e96f0fc01-0'),
 AIMessage(content='Why did Shane Warne always bring a spare ball to the cricket match?\n\nBecause he knew he\'d "bowl" over the opposition! 😜 \n', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXU

<h3>Event Reference</h3>

In [59]:
template="""
Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt=ChatPromptTemplate.from_template(template=template)

vectorStore=FAISS.from_texts(
    texts=["Ritish has a 2.5 month old son !",
           "Rohit is Ritish's Younger Brother",
           "Eklavya is Ritish's Son"
           "Rekha is the name of Ritish's Mother",
           "Anirban is Ritish's Father"],
    embedding=SentenceTransformerEmbeddings())

retriever=vectorStore.as_retriever()

outputParser=StrOutputParser()

In [60]:
retrievalChain=RunnablePassthrough.assign(context=itemgetter("question")|retriever.with_config(run_name="Docs") ) | prompt | llm.with_config(run_name="my_llm") | outputParser

In [64]:
question={"question":"Who is Anirban's Wife and who is Eklavya's Uncle?"}

In [66]:
print(retrievalChain.invoke(input=question))

Here's the breakdown based on the provided context:

* **Anirban's Wife:**  The context states "Rekha is the name of Ritish's Mother". Since Anirban is Ritish's father, **Rekha is Anirban's wife**. 
* **Eklavya's Uncle:** We know Ritish has a 2.5 month old son. We can infer that this son is named Eklavya.  Ritish is Eklavya's father, and Rohit is Ritish's brother. Therefore, **Rohit is Eklavya's uncle**. 



In [67]:
async for event in retrievalChain.astream_events(input=question,
                                                 version="v1",
                                                 include_names=["Docs","my_llm"]
                                                ):
    print(event,"\n")

{'event': 'on_retriever_start', 'name': 'Docs', 'run_id': '3f5917f3-d872-483e-a07e-a5b87a7e70cd', 'tags': ['seq:step:2', 'FAISS', 'HuggingFaceEmbeddings'], 'metadata': {'ls_retriever_name': 'vectorstore', 'ls_vector_store_provider': 'FAISS', 'ls_embedding_provider': 'HuggingFaceEmbeddings'}, 'data': {'input': {'query': "Who is Anirban's Wife and who is Eklavya's Uncle?"}}, 'parent_ids': []} 

{'event': 'on_retriever_end', 'name': 'Docs', 'run_id': '3f5917f3-d872-483e-a07e-a5b87a7e70cd', 'tags': ['seq:step:2', 'FAISS', 'HuggingFaceEmbeddings'], 'metadata': {'ls_retriever_name': 'vectorstore', 'ls_vector_store_provider': 'FAISS', 'ls_embedding_provider': 'HuggingFaceEmbeddings'}, 'data': {'input': {'query': "Who is Anirban's Wife and who is Eklavya's Uncle?"}, 'output': {'documents': [Document(page_content="Anirban is Ritish's Father"), Document(page_content="Rekha is the name of Ritish's Mother"), Document(page_content="Rohit is Ritish's Younger Brother"), Document(page_content='Ritish 

In [68]:
async for event in retrievalChain.astream_events(input=question,
                                                 version="v1",
                                                 include_names=["Docs","my_llm"]
                                                ):
    kind=event['event']
    if kind=="on_chat_model_stream":
        print(event["data"]["chunk"].content,end="|")
    elif kind=="on_chat_model_start":
        print("\nStreaming LLM")
    elif kind=="on_chat_model_end":
        print("Done Streaming LLM.")
    elif kind=="on_retriever_end":
        print("Retrieved the following Documents:")
        print(event["data"]["output"]["documents"])
    elif kind=="on_tool_end":
        print(f"Ended Tool: {event['name']}")

Retrieved the following Documents:
[Document(page_content="Anirban is Ritish's Father"), Document(page_content="Rekha is the name of Ritish's Mother"), Document(page_content="Rohit is Ritish's Younger Brother"), Document(page_content='Ritish has a 2.5 month old son !')]

Streaming LLM
Here's the| breakdown based on the provided context:

* **Anirban's Wife:**|  The context states "Rekha is the name of Ritish's Mother|". Since Anirban is Ritish's father, **Rekha is Anirban's wife**. 
* **Eklavya's| Uncle:** The context mentions "Ritish has a 2.5 month old son!". We can infer that Ritish's son is named Eklavya.|  Since Rohit is Ritish's younger brother, **Rohit is Eklavya's uncle**. 
|Done Streaming LLM.


In [70]:
async for chunk in retrievalChain.astream_log(input=question,include_names=["Docs"]):
    print("-"*40)
    print(chunk)

----------------------------------------
RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': '2fc15f6d-8774-4034-b849-82439898aafc',
            'logs': {},
            'name': 'RunnableSequence',
            'streamed_output': [],
            'type': 'chain'}})
----------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/Docs',
  'value': {'end_time': None,
            'final_output': None,
            'id': '687397bd-b966-470a-84a9-9e6b2e327be6',
            'metadata': {'ls_embedding_provider': 'HuggingFaceEmbeddings',
                         'ls_retriever_name': 'vectorstore',
                         'ls_vector_store_provider': 'FAISS'},
            'name': 'Docs',
            'start_time': '2024-09-24T20:41:52.687+00:00',
            'streamed_output': [],
            'streamed_output_str': [],
            'tags': ['seq:step:2', 'FAISS', 'HuggingFaceEmbeddings'],
            'type': 'retriever'}})
---------------

In [71]:
## Stream Incremental RunState with diff=False
async for chunk in retrievalChain.astream_log(input=question,include_names=["Docs"],diff=False):
    print("-"*40)
    print(chunk)

----------------------------------------
RunLog({'final_output': None,
 'id': '285c7584-ffc6-4fda-8de4-3a2a96530f6c',
 'logs': {},
 'name': 'RunnableSequence',
 'streamed_output': [],
 'type': 'chain'})
----------------------------------------
RunLog({'final_output': None,
 'id': '285c7584-ffc6-4fda-8de4-3a2a96530f6c',
 'logs': {'Docs': {'end_time': None,
                   'final_output': None,
                   'id': 'd6cba482-cd44-4f72-bab0-7fe738b5d482',
                   'metadata': {'ls_embedding_provider': 'HuggingFaceEmbeddings',
                                'ls_retriever_name': 'vectorstore',
                                'ls_vector_store_provider': 'FAISS'},
                   'name': 'Docs',
                   'start_time': '2024-09-24T20:44:13.157+00:00',
                   'streamed_output': [],
                   'streamed_output_str': [],
                   'tags': ['seq:step:2', 'FAISS', 'HuggingFaceEmbeddings'],
                   'type': 'retriever'}},
 'name':

<h3>Runnable Parallel</h3>

In [5]:
chain1=ChatPromptTemplate.from_template(template="Tell Me a Joke about {topic}")|llm
chain2=ChatPromptTemplate.from_template(template="Write a Poem on the Following {subject}")|llm

In [6]:
combinedChain=RunnableParallel(joke=chain1,poem=chain2)

In [7]:
start=time.time()
print(chain1.invoke(input={"topic":"Titanic"}).content)
print(f"Took {time.time()-start} Seconds")

Why did the Titanic sink? 

Because it couldn't find a parking space! 

Took 1.872040033340454 Seconds


In [8]:
start=time.time()
print(chain2.invoke(input={"subject":"Titanic"}).content)
print(f"Took {time.time()-start} Seconds")

A steel behemoth, a dream come true,
The Titanic, a marvel, grand and new.
From Belfast's shores, she sailed with pride,
A symbol of progress, on the ocean tide.

Her decks were bustling, a vibrant scene,
With passengers of every rank and glean.
From wealthy tycoons to families bold,
Each with their stories, yet to unfold.

The band played melodies, sweet and clear,
As the ship glided, banishing all fear.
But fate had a cruel, and icy hand,
A chilling whisper, across the land.

An iceberg loomed, in the starlit night,
A collision fatal, a tragic plight.
The ship began to sink, a mournful cry,
As hope dwindled, beneath the sky.

Lifeboats strained, with souls in despair,
As the Titanic, met her final prayer.
The ocean claimed her, in its cold embrace,
Leaving behind, a haunting trace.

A monument to ambition's might,
A reminder of loss, in the darkest night.
The Titanic's story, forever told,
Of pride and tragedy, both brave and bold. 

Took 1.7491569519042969 Seconds


In [10]:
start=time.time()
print(combinedChain.invoke(input={"subject":"Titanic","topic":"Titanic"}))
print(f"Took {time.time()-start} Seconds")

{'joke': AIMessage(content="Why did the Titanic sink?\n\nBecause it couldn't find a parking space in the iceberg lot! \n", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-609d15f6-c612-4f8e-a794-26b63ce8f5a7-0', usage_metadata={'input_tokens': 7, 'output_tokens': 21, 'total_tokens': 28}), 'poem': AIMessage(content="A steel leviathan, a dream of might,\nThe Titanic, bathed in moonlight's light.\nAcross the ocean, a majestic stride,\nA ship of dreams, with nowhere to hide.\n\nHer decks alive with laughter, song, and 

In [11]:
RunnableParallel({'joke':chain1,'poem':chain2}).invoke(input={"subject":"Titanic","topic":"Titanic"})


{'joke': AIMessage(content="Why didn't the Titanic sink faster? \n\nBecause it was full of holes! \n", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-ac5cba43-736d-4351-882a-5992a2c6f37c-0', usage_metadata={'input_tokens': 7, 'output_tokens': 18, 'total_tokens': 25}),
 'poem': AIMessage(content="A steel leviathan, a dream of might,\nThe Titanic sailed, a beacon bright.\nA grand facade, a promise bold,\nOf luxury and stories yet untold.\n\nAcross the ocean, a majestic glide,\nWith passengers aboard, their hopes ins

<h3> Parallelism on Batches</h3>

In [83]:
start=time.time()
print(chain1.batch(inputs=[{"topic":"Titanic"},{"topic":"Kolkata"}]))
print(f"Took {time.time()-start} Seconds")

[AIMessage(content="Why did the Titanic sink? \n\nBecause it couldn't find a parking space in the iceberg lot! \n", response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-f01b7aa5-e956-4af4-8bcb-df0bdfd70625-0'), AIMessage(content='Why did the Bengali man cross the road? \n\nTo get to the other side of the **ghat**! \n\n(Get it? Ghat is a riverbank, a common feature in Kolkata.) \n', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_

In [84]:
start=time.time()
print(chain2.batch(inputs=[{"subject":"Titanic"},{"subject":"Kolkata"}]))
print(f"Took {time.time()-start} Seconds")

[AIMessage(content="A steel leviathan, a dream of might,\nThe Titanic, gleaming, bathed in light.\nA promise of a journey, swift and grand,\nAcross the ocean, to a promised land.\n\nHer decks, a bustling city, life in bloom,\nWith laughter, music, chasing away the gloom.\nThe wealthy, the hopeful, all aboard,\nA tapestry of stories, each word adored.\n\nBut fate, a cruel and silent hand,\nHad spun a different thread, a tragic plan.\nAn iceberg, looming, cold and stark,\nA fatal collision, leaving its dark mark.\n\nThe ocean roared, a symphony of dread,\nAs icy water filled the ship, instead\nOf warmth and hope, a chilling, icy tomb,\nA final waltz, played by the ocean's boom.\n\nFrom gilded halls to steerage, all in fear,\nThe cries of anguish, echoing far and near.\nLifeboats filled, leaving souls behind,\nA tragic spectacle, etched in mankind's mind.\n\nThe Titanic sank, a legend born,\nA symbol of ambition, tragically torn.\nA reminder of the fragile human plight,\nOf dreams shatter

In [85]:
start=time.time()
print(combinedChain.batch(inputs=[
                                    {"subject":"Titanic","topic":"Titanic"},  # Joke on Titanic and Poem on Titanic
                                    {"subject":"Kolkata","topic":"Kolkata"}  # Joke on Kolkata and Poem on Kolkata
                          ]))
print(f"Took {time.time()-start} Seconds")

[{'joke': AIMessage(content="Why did the Titanic sink? \n\nBecause it couldn't find a parking space at the bottom of the ocean! \n", response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-2ba715f8-ddd2-40f5-a3bb-f24864b95dc7-0'), 'poem': AIMessage(content="A steel leviathan, a dream of might,\nThe Titanic, a beacon in the night.\nAcross the waves, she sailed with grace,\nA gilded cage, a fleeting space.\n\nHer decks a haven, filled with glee,\nOf laughter, music, destiny.\nThe band played on, a mournful tune,\nAs fate's dark hand began to loom

In [89]:
combinedChain.get_graph().print_ascii()

                       +--------------------------+                        
                       | Parallel<joke,poem>Input |                        
                       +--------------------------+                        
                       *****                   *****                       
                    ***                             ***                    
                 ***                                   ***                 
  +--------------------+                          +--------------------+   
  | ChatPromptTemplate |                          | ChatPromptTemplate |   
  +--------------------+                          +--------------------+   
             *                                               *             
             *                                               *             
             *                                               *             
+------------------------+                      +------------------------+ 
| ChatGoogle

In [90]:
combinedChain.get_prompts()

[ChatPromptTemplate(input_variables=['topic'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], template='Tell Me a Joke about {topic}'))]),
 ChatPromptTemplate(input_variables=['subject'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['subject'], template='Write a Poem on the Following {subject}'))])]